In [2]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [5]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-v3rb9y9p
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-v3rb9y9p
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=6699669c1c4bdf9323f8252b02913e534763f257b0508a742931669c9f073969
  Stored in directory: /tmp/pip-ephem-wheel-cache-83ajujmy/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [6]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void add(float *a, float *b, float *c, int n) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    printf("~~~%d~~~\n", i);
    if(i < n)
        c[i] = a[i] + b[i];
}

int main() {
int n = 16*16;
int dev_count;
cudaGetDeviceCount(&dev_count);
float *h_a, *h_b, *h_c;
float *d_a, *d_b, *d_c;
int size = n*sizeof(float);
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
h_a = (float *) malloc(n*sizeof(float));
h_b = (float *) malloc(n*sizeof(float));
h_c = (float *) malloc(n*sizeof(float));

for(int i=0; i<n; i++)
{
   h_a[i] = i+1;
   h_b[i] = i+2;  
}

dim3 gridDim(n/16,1,1);
dim3 blockDim(16,1,1);

cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);
add<<<gridDim, blockDim>>>(d_a, d_b, d_c, n);
cudaError err = cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
for(int i=0; i < n; i++)
{
    printf("%d value : %f, %f, %f\n", i, h_a[i], h_b[i], h_c[i]);
}
//printf("dev_cnt is %d and result is %d\n",dev_count, c);
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}

In [41]:
%%cu

#include <stdio.h>
#include <stdlib.h>

int main()
{
    cudaDeviceProp dev_prop;
    int *x = new int(5);
    int dev_cnt;
    cudaGetDeviceCount(&dev_cnt);
    
    for(int i=0; i<dev_cnt; i++)
    {
        printf("Device %d :\n", i);
        cudaGetDeviceProperties(&dev_prop, i);
        printf("dev_prop.maxThreadsPerBlock : %d\ndev_prop.multiProcessorCount : %d\ndev_prop.clockRate : %d\ndev_prop.maxThreadsDim[0] : %d\ndev_prop.maxGridSize[0] : %d\n",
               dev_prop.maxThreadsPerBlock, dev_prop.maxThreadsPerBlock, dev_prop.clockRate, dev_prop.maxThreadsDim[0], dev_prop.maxGridSize[0]);
    }
    return 0;
}

Device 0 :
dev_prop.maxThreadsPerBlock : 1024
dev_prop.multiProcessorCount : 1024
dev_prop.clockRate : 823500
dev_prop.maxThreadsDim[0] : 1024
dev_prop.maxGridSize[0] : 2147483647



In [35]:
!lshw -c display

/bin/bash: lshw: command not found
